In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import os
import sys
import pandas as pd
import numpy as np
import quandl
import cvxpy as cvx

In [3]:
sys.path.insert(0, os.path.abspath('..'))
import helium

In [4]:
%matplotlib inline

In [5]:
quandl.ApiConfig.api_key = '8wnB7EcvJm1xEvJjzWBR'

In [6]:
tickers = ['AMZN', 'GOOGL', 'MCD', 'NKE']
start_date='2016-01-01'
end_date='2018-02-01'
prices = pd.DataFrame(dict([(ticker, quandl.get('WIKI/'+ticker, 
                                    start_date=start_date, 
                                    end_date=end_date)['Adj. Close'])
                for ticker in tickers]))

returns=prices.pct_change()
returns[["_CASH"]]=quandl.get('FRED/DTB3', start_date=start_date, end_date=end_date)/(250*100)
returns = returns.fillna(method='ffill').iloc[1:]

returns.tail()

,AMZN,GOOGL,MCD,NKE,_CASH
Date,,,,,
2018-01-26,0.017490,0.004585,0.015371,0.004874,0.000056
2018-01-29,0.011148,-0.000909,-0.003308,-0.006761,0.000057
2018-01-30,0.014206,-0.007678,-0.029758,-0.003699,0.000057
2018-01-31,0.009619,0.004119,-0.007769,0.013218,0.000058
2018-02-01,-0.042469,-0.000533,0.004441,-0.008355,0.000058


In [7]:
rets = returns.rolling(window=250, min_periods=250).mean().shift(1).dropna()
sigmas = returns.rolling(window=250, min_periods=250).cov().unstack().shift(1).stack()

rets.tail()

,AMZN,GOOGL,MCD,NKE,_CASH
Date,,,,,
2018-01-26,0.002075,0.001334,0.001574,0.001069,0.00004
2018-01-29,0.002161,0.001409,0.001603,0.001123,0.00004
2018-01-30,0.002231,0.001505,0.001584,0.001104,0.00004
2018-01-31,0.002321,0.001492,0.001480,0.001103,0.00004
2018-02-01,0.002317,0.001533,0.001454,0.001146,0.00004


In [8]:
cash_ticker = "_CASH"

In [9]:
price_sigma = rets.copy()
price_sigma[:] = 0.

In [10]:
volumes = pd.DataFrame(dict([(ticker, quandl.get('WIKI/'+ticker, 
                                    start_date=start_date, 
                                    end_date=end_date)['Adj. Volume'])
                for ticker in tickers]))
volumes[cash_ticker] = np.finfo(np.float).max

In [11]:
tcost_model=helium.TransactionCost(gamma = 1., 
                                   half_spread=10E-4,
                                   nonlin_coef = 0.,  #1.,
                                   sigmas = price_sigma, 
                                   nonlin_power = 1.5,
                                   volumes = volumes,
                                   asym_coef = 0.)

In [12]:
borrow_costs = rets.copy()
borrow_costs[:] = 1E-4
borrow_costs["_CASH"] = 0.

dividends = rets.copy()
dividends[:] = 0.


hcost_model=helium.HoldingCost(gamma = 1., borrow_costs=borrow_costs, dividends=dividends)

In [13]:
risk_model = helium.BasicRiskCost(5.0, sigmas)

In [14]:
leverage_limit = helium.LeverageLimitConstraint(3.)

In [15]:
holding_init = pd.Series(index=rets.columns, data=0.)
holding_init[cash_ticker] = 10000.
holding_init

AMZN         0.0
GOOGL        0.0
MCD          0.0
NKE          0.0
_CASH    10000.0
dtype: float64

In [16]:
deltas = rets.copy()
deltas[:]  = 0.

In [17]:
r = helium.DefaultRet(rets=rets, deltas = deltas, gamma_decay=None)

In [18]:
spo_policy = helium.SinglePeriodOpt(rets=r,
                                   costs=[risk_model, hcost_model, tcost_model],
                                   constraints=[leverage_limit]
                                   )

In [19]:
today = pd.datetime(2018, 2, 1)

In [20]:
shares_to_trade=spo_policy.get_trades(h=holding_init, t=today)
shares_to_trade

******
QQQ: AMZN         0.0
GOOGL        0.0
MCD          0.0
NKE          0.0
_CASH    10000.0
dtype: float64
Obj: sum_entries(mul_elemwise([[  2.31664660e-03]
 [  1.53280288e-03]
 [  1.45386101e-03]
 [  1.14645450e-03]
 [  4.03712000e-05]], [[ 0.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 1.]] + var0)) + -0 + 5.0 * 0 + 0.000234498065008 * quad_over_lin([[ -7.68551789e-08   5.68652273e-08   7.00930604e-08  -2.52375897e-08
    6.08620095e-04]
 [  2.05649227e-01  -3.50162519e-01   4.03484090e-02   1.09466205e-02
    5.44927821e-05]
 [  9.70433997e-02  -3.90305915e-03  -5.27894083e-01  -2.18540614e-03
    7.33245485e-05]
 [ -6.01651846e-02  -6.47561850e-02  -6.78888172e-03  -9.16113097e-01
   -3.87536724e-05]
 [  8.42181589e-01   5.09170788e-01   1.51437245e-01  -9.24231514e-02
    3.75023215e-05]] * [[ 0.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 1.]] + var0, 1) + 1.0 * sum_entries(mul_elemwise([[ 0.0001]
 [ 0.0001]
 [ 0.0001]
 [ 0.0001]], --max_elemwise(-[[ 0.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 1.]] + var0[:-1, 0], -0)) + -mul_e

AMZN      6698.883895
GOOGL        0.000234
MCD       3951.501392
NKE        726.158527
_CASH   -11376.544048
dtype: float64

In [21]:
np.round(shares_to_trade/prices.loc[today])

AMZN      5.0
GOOGL     0.0
MCD      23.0
NKE      11.0
_CASH     NaN
dtype: float64